<a href="https://colab.research.google.com/github/kriskalb/flight-delay-prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import joblib
import pandas as pd
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, mean_squared_error
import pyarrow.parquet as pq
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


# Regression - ARRIVAL_DELAY prediction - xgboost

In [6]:
parquet_file_path = '/content/drive/MyDrive/Kurs/final_project/inputs/df_selected.parquet'
table = pq.read_table(parquet_file_path)
df_selected = table.to_pandas()
parquet_file_path = '/content/drive/MyDrive/Kurs/final_project/inputs/df_target.parquet'
table = pq.read_table(parquet_file_path)
df_target = table.to_pandas()

In [8]:
xgboost_model = joblib.load('/content/drive/MyDrive/Kurs/final_project/models/xgboost.pkl')

In [12]:
X = df_selected
y = df_target['ARRIVAL_DELAY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
y_pred_xgboost = xgboost_model.predict(X_test)

In [15]:
mse_zgb = mean_squared_error(y_test, y_pred_xgboost)
mae_xgb = mean_absolute_error(y_test, y_pred_xgboost)
print(f"Mean Squared Error: {mse_zgb}")
print(f"Mean Absolute Error: {mae_xgb}")

Mean Squared Error: 154.3069469350176
Mean Absolute Error: 8.683368195024757


# Classification - ARRIVAL_DELAY (True/False) prediction - logistic regression

In [16]:
parquet_file_path = '/content/drive/MyDrive/Kurs/final_project/inputs/df_selected_classification.parquet'
table = pq.read_table(parquet_file_path)
df_selected_classification = table.to_pandas()
parquet_file_path = '/content/drive/MyDrive/Kurs/final_project/inputs/df_target_encoded.parquet'
table = pq.read_table(parquet_file_path)
df_target_encoded = table.to_pandas()

In [17]:
X = df_selected_classification
y = df_target_encoded['ARRIVAL_DELAY_encoded']

In [18]:
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X, y, test_size=0.2, random_state=42)
columns_to_standarize = ['SCHEDULED_TIME', 'DISTANCE', 'ORIG_LATITUDE', 'ORIG_LONGITUDE', 'DEST_LONGITUDE'
                    ,'ORIGIN_AIRPORT_RANK',
                    'DESTIN_AIRPORT_RANK',]
scaler = StandardScaler()
X_train_lr[columns_to_standarize] = scaler.fit_transform(X_train_lr[columns_to_standarize])
X_test_lr[columns_to_standarize] = scaler.transform(X_test_lr[columns_to_standarize])

In [19]:
lr_model = joblib.load('/content/drive/MyDrive/Kurs/final_project/models/logistic.pkl')

In [20]:
y_pred_lr = lr_model.predict(X_test_lr)

In [21]:
accuracy = accuracy_score(y_test_lr, y_pred_lr)
print(f"Accuracy- random forest: {accuracy}")

classification_rep = classification_report(y_test_lr, y_pred_lr)
print("classification report:")
print(classification_rep)

Accuracy- random forest: 0.6399663297083823
classification report:
              precision    recall  f1-score   support

           0       0.64      0.99      0.78    655853
           1       0.75      0.05      0.09    388391

    accuracy                           0.64   1044244
   macro avg       0.69      0.52      0.43   1044244
weighted avg       0.68      0.64      0.52   1044244

